## Normalized Design
> In a normalized database, the relationships among the tables match the relationships that are really there among the data.

###### 1. Every row has the same number of columns.
###### 2. There is a [unique] key, and everything in a row says something about the key.
###### 3. Facts that don't relate to the key belong in different tables
###### 4. Tables shouldn't imply relationships that don't exist

In [ ]:
# create table and types
create table TABLENAME(
    column1 type [constraints],
    column2 type [constraints],
    ……
    [row constraints]);

In [ ]:
# creating and dropping
create database name[options];
drop database name[options];
drop table name[options];
# postgreSQL docs.

### primary key:
> a column or columns that uniquely identify what each row in a table is about

In [ ]:
# declaring primary keys
create table students(
    id serial primary key,
    name text,
    birthdate date
);
# multi-column primary key
create table postal_places(
    postal_code text,
    country text,
    name text,
    primary key(postal_code,country)
);

### Reference
> references provides referential integrity --columns that are supposed to refer to each other are guaranteed to do so.

In [ ]:
# Declaring Relationships
create table sales(
    sku text references products(sku),
    sale_date date,
    count integer);


### Foreign Keys
> A foreign key is a column or set of columns in one table, that uniquely identifies rows in another table.

In [ ]:
create table students(
    id serial primary key,
    name text);
create table courses(
    id text primary key,
    name text);
create table grades(
    student integer references students(id),
    course text references courses(id),
    grade text)

In [ ]:
# Self Joins
select a.id, b.id
from residences as a,
     residences as b
where
    a.building = b.building
and a.room = b.room 

In [ ]:
#
# Roommate Finder v0.9
#
# This query is intended to find pairs of roommates.  It almost works!
# There's something not quite right about it, though.  Find and fix the bug.
#

QUERY = '''
select a.id, b.id, a.building, a.room
       from residences as a, residences as b
 where a.building = b.building
   and a.room = b.room
   and a.id < b.id     

 order by a.building, a.room;
'''

#
# To see the complete residences table, uncomment this query and press "Test Run":
#
# QUERY = "select id, building, room from residences;"
# 

# Count

```SQL
SELECT count(*) FROM animals; 
#return the amount of animals
SELECT count(*) FROM animals
WHERE species = 'gorilla';
#return the amount of gorilla
SELECT species,count(*) from animals 
GROUP BY species;
#return species and corresponding amount of
each speices
```


# Count non-existed value as 0

In [ ]:
# In this quiz, there's a table describing bugs in various files of code.
# Here's what the table looks like:
#
# create table programs (
#    name text,
#    filename text
# );
# create table bugs (
#    filename text,
#    description text,
#    id serial primary key
# );
#
# The query below is intended to count the number of bugs in each program. But
# it doesn't return a row for any program that has zero bugs.  Try running it as
# it is; then change it so that it includes rows for the programs with no bugs.

QUERY = '''
select programs.name, count(bugs.filename) as num
   from programs left join bugs
        on programs.filename = bugs.filename
   group by programs.name
   order by num;
'''


>注意事项：在 count 聚合函数中放入哪些参数？如果保留 count(*) <br>
不变或者将 programs 表中的某一列作为其参数，查询将对有缺陷和无缺陷的程序进行计数。

>要为在 bugs 表中没有任何条目的程序报告零值，则必须将 bugs 表中的列作为 count 的参数。例如，count(bugs.filename) 和 count(bugs.description) 均具有此作用。

# DB-API

In [ ]:
# To see how the various functions in the DB-API work, take a look at this code,
# then the results that it prints when you press "Test Run".
#
# Then modify this code so that the student records are fetched in sorted order
# by student's name.
#

import sqlite3

# Fetch some student records from the database.
db = sqlite3.connect("students")
c = db.cursor()
query = "select name, id from students;"
c.execute(query)
rows = c.fetchall()

# First, what data structure did we get?
print "Row data:"
print rows

# And let's loop over it too:
print
print "Student names:"
for row in rows:
    print "  ", row[0]

db.close()


In [ ]:
import sqlite3
conn = sqlite3.connect("Cookies")
cursor = conn.cursor()
# The cursor is what actually runs queries and fetches results
# It's called a cursor because when the database gives you results
# you use the cursor to scan through the results, kind of like a 
# text cursor in an editor

cursor.execute(
"SELECT host_key FROM Cookies limit 10")

# execute a query using a cursor,
results = cursor.fetchall()
# fetchall the results from that query also using the cursor

print(results)
conn.close()

In [ ]:
#Inserts in DB-API
pg = psycopg2.connect("dbname=somedb")
c = pg.cursor()
c.execute("insert into names values('Jennifer')")
pg.commit()
# by adding commit, you can make sure that your 
# changes get saved.
# one nice thing about transactions and rollbacks
# is that your code crashes in the middle of a database
# transaction, you can be sure that it won't have writte
# half-finished changes.

atomicity
> A transaction happens as a whole or not at all.

# Subquery
##### Average high-scorer's score:
```SQL
SELECT avg(bigscore)
FROM
    (SELECT max(score) as bigscore
    FROM mooseball
    GROUP BY team) as maxes
```
    # the syntax of sql requires that we give
    the subquery result table a name, (maxes)

```SQL
SELECT name,weight FROM players,
    (SELECT avg(weight) as av
    FROM plyers) as subq 
WHERE weight < av;
```

# Views
> A view is a select query stored in the database in a way that lets you use it like a table 
<br>(refactor the tough query)

### create view VIEWNAME as select...
       view are really useful with aggregations like counts or sums
       
### How many students are enrolled in each course? (table enrollment with column student_id and course_id)
```SQL
create view course_size as
SELECT course_id,count(*) as num
FROM enrollment
GROUP BY course_id;
```